In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.68" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -c -O 'nature_12K.zip'

--2021-04-15 05:10:50--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 172.217.7.208, 172.217.164.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.9.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  36.9MB/s    in 65s     

2021-04-15 05:11:56 (55.6 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
!unzip "/content/nature_12K.zip"

In [3]:
import shutil
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPooling2D,Activation,Dropout,Flatten,BatchNormalization
from tensorflow.keras.models import Model,Sequential
import random 
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

In [4]:
class CNN:
  def __init__(self,no_of_filters=16,size_of_filters=(3,3),activation_func="relu",neurons_in_dense_layer=128
               ,dropout=0.3,batch_norm=False,doubling=False,augmentation=False):
    self.augmentation = augmentation
    self.dropout = dropout
    self.doubling = doubling
    self.batch_norm = batch_norm
    self.no_of_filters = no_of_filters
    self.size_of_filters = size_of_filters
    self.activation_func = activation_func
    self.neurons_in_dense_layer = neurons_in_dense_layer
    self.train_path = "/content/inaturalist_12K/train/"
    self.cv_path = "/content/inaturalist_12K/cv/"
    self.test_path = "/content/inaturalist_12K/val/"
    
  def generate_data(self):
    if self.augmentation:
      train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True)
    else:
      train_datagen = ImageDataGenerator(rescale=1./255)
    cv_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    self.train_generator = train_datagen.flow_from_directory(self.train_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.cv_generator = cv_datagen.flow_from_directory(self.cv_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.test_generator = test_datagen.flow_from_directory(self.test_path,target_size=(224,224),seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")

  def create_cv_data(self):
    classes = sorted(os.listdir(self.train_path))[1:]
    os.mkdir(self.cv_path)
    for cls in classes:
      os.mkdir(self.cv_path+cls)

    for cls in classes:
      no_of_files = len(os.listdir(self.train_path+cls))
      files = os.listdir(self.train_path+cls)
      files = np.asarray(files)

      x = [i for i in range(no_of_files)]
      random.shuffle(x)
      index = x[:int(0.1*no_of_files)]
      file_names = files[index]

      for z in file_names:
        shutil.move(self.train_path+cls+"/"+z , self.cv_path+cls+"/"+z)

  def create_model(self):
    self.generate_data()
    model = Sequential()
    random.seed(10)
    for i in range(5):
      if i==0:
        model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation= self.activation_func, input_shape=(224,224,3)))
      else:
        if self.doubling:
          model.add(Conv2D(self.no_of_filters*(2**i), self.size_of_filters ,padding="same",activation = self.activation_func))
        else:
          model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation = self.activation_func))
      
      if i!=0 and i%2 == 0 and self.batch_norm:
        model.add(BatchNormalization())
      model.add(MaxPooling2D(pool_size=(2, 2))) 

    model.add(Flatten())
    model.add(Dropout(self.dropout))
    model.add(Dense(self.neurons_in_dense_layer,activation = self.activation_func))
    if self.batch_norm:
      model.add(BatchNormalization())
    model.add(Dropout(self.dropout))
    model.add(Dense(10,activation='softmax'))
    return model


In [5]:
cnn = CNN(no_of_filters=32,neurons_in_dense_layer=64,dropout = 0.2
              ,batch_norm=False,doubling=False,augmentation=False)
cnn.create_cv_data()

model = cnn.create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0008),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(cnn.train_generator, validation_data = cnn.cv_generator,epochs=6)

Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Epoch 1/6
141/141 [==============================] - 151s 842ms/step - loss: 2.2878 - accuracy: 0.1198 - val_loss: 2.1125 - val_accuracy: 0.2322
Epoch 2/6
141/141 [==============================] - 113s 804ms/step - loss: 2.1295 - accuracy: 0.2190 - val_loss: 2.0287 - val_accuracy: 0.2693
Epoch 3/6
141/141 [==============================] - 106s 751ms/step - loss: 2.0311 - accuracy: 0.2713 - val_loss: 2.0052 - val_accuracy: 0.2963
Epoch 4/6
141/141 [==============================] - 107s 759ms/step - loss: 1.9879 - accuracy: 0.2898 - val_loss: 1.9198 - val_accuracy: 0.3173
Epoch 5/6
141/141 [==============================] - 110s 781ms/step - loss: 1.9037 - accuracy: 0.3187 - val_loss: 1.8745 - val_accuracy: 0.3373
Epoch 6/6
141/141 [==============================] - 107s 762ms/step - loss: 1.8629 - accuracy: 0.3301 - val_loss: 1.8534 - val_accuracy: 0.3363


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        9

In [7]:
model.fit(cnn.train_generator,validation_data=cnn.test_generator)

141/141 [==============================] - 120s 849ms/step - loss: 1.8271 - accuracy: 0.3650 - val_loss: 1.8353 - val_accuracy: 0.3595
